In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

from thor_magni_tools.utils.load import load_csv_metadata_magni, preprocessing_header_magni
from thor_magni_tools.preprocessing.filtering import Filterer3DOF
from thor_magni_tools.preprocessing import TrajectoriesReprocessor
from thor_magni_tools.utils.visualization_nb import visualize_scenario

In [52]:
DIR_PATH = "../datasets/thor_magni_zenodo/"
SCENARIO_ID = "Scenario_2"

In [53]:
raw_df, header_dict = load_csv_metadata_magni(
    os.path.join(DIR_PATH, SCENARIO_ID, "THOR-Magni_130522_SC2_R1.csv")
)

In [54]:
new_header_dict = preprocessing_header_magni(header_dict)
traj_metadata = new_header_dict["SENSOR_DATA"]["TRAJECTORIES"]["METADATA"]

# Filtering markers

In [55]:
roles = {k: metadata["ROLE"] for k, metadata in traj_metadata.items()}
best_markers_traj = Filterer3DOF.filter_best_markers(raw_df, roles)

In [56]:
restored_markers = Filterer3DOF.restore_markers(raw_df, roles)

In [57]:
restored_markers

,frame_id,ag_id,x,y,z,data_label
Time,,,,,,
0.01,2,Helmet_1,-126.937333,3293.304333,1766.387000,Visitors-Group 2
0.01,2,Helmet_5,8693.070500,350.301000,1877.664000,Carrier- Large Object Leader
0.01,2,LO1,-499.704400,-3128.781000,1483.196000,Carried
0.01,2,Helmet_10,-7901.285000,1364.663000,1588.384000,Carrier-Bucket
0.01,2,Helmet_4,NaN,NaN,NaN,Carrier- Large Object Follower
...,...,...,...,...,...,...
242.94,24295,DARKO_Robot,-4362.402857,-1344.465429,606.237000,Obstacle
242.94,24295,Helmet_6,-706.726200,2773.144400,1798.646600,Visitors-Group 2
242.94,24295,Helmet_1,-3.950333,3056.111000,1834.424333,Visitors-Group 2


In [58]:
def pairwise_distances(points):
    x, y = points[:, 0], points[:, 1]
    return np.sqrt((x[:, None] - x) ** 2 + (y[:, None] - y) ** 2)

In [59]:
restored_markers

,frame_id,ag_id,x,y,z,data_label
Time,,,,,,
0.01,2,Helmet_1,-126.937333,3293.304333,1766.387000,Visitors-Group 2
0.01,2,Helmet_5,8693.070500,350.301000,1877.664000,Carrier- Large Object Leader
0.01,2,LO1,-499.704400,-3128.781000,1483.196000,Carried
0.01,2,Helmet_10,-7901.285000,1364.663000,1588.384000,Carrier-Bucket
0.01,2,Helmet_4,NaN,NaN,NaN,Carrier- Large Object Follower
...,...,...,...,...,...,...
242.94,24295,DARKO_Robot,-4362.402857,-1344.465429,606.237000,Obstacle
242.94,24295,Helmet_6,-706.726200,2773.144400,1798.646600,Visitors-Group 2
242.94,24295,Helmet_1,-3.950333,3056.111000,1834.424333,Visitors-Group 2


In [64]:
restored_markers_helmets = restored_markers[~restored_markers.ag_id.str.startswith(("DARKO", "LO"))]

In [65]:
restored_markers_helmets

,frame_id,ag_id,x,y,z,data_label
Time,,,,,,
0.01,2,Helmet_1,-126.937333,3293.304333,1766.387000,Visitors-Group 2
0.01,2,Helmet_5,8693.070500,350.301000,1877.664000,Carrier- Large Object Leader
0.01,2,Helmet_10,-7901.285000,1364.663000,1588.384000,Carrier-Bucket
0.01,2,Helmet_4,NaN,NaN,NaN,Carrier- Large Object Follower
0.01,2,Helmet_3,-985.657750,-3656.280250,1375.076750,Visitors-Alone
...,...,...,...,...,...,...
242.94,24295,Helmet_3,-8624.673500,370.725000,1735.359000,Visitors-Alone
242.94,24295,Helmet_10,-5271.148750,999.940750,1691.664250,Carrier-Bucket
242.94,24295,Helmet_6,-706.726200,2773.144400,1798.646600,Visitors-Group 2


In [62]:
grouped_frames = restored_markers_helmets.groupby("Time")

In [45]:
distances, min_distances = {}, []
for time, group in grouped_frames:
    distances[time], distances_ts = [], []
    points = group[["x", "y"]].values
    agents_ids = group["ag_id"].values
    pairwise_dist_matrix = pairwise_distances(points)
    np.fill_diagonal(pairwise_dist_matrix, np.inf)
    agents_combinations = list(combinations(range(len(agents_ids)), 2))
    ts_distances = []
    for i, j in agents_combinations:
        if not np.isnan(pairwise_dist_matrix[i, j]):
            distances[time].append(
                {
                    "ag_id1": agents_ids[i],
                    "ag_id2": agents_ids[j],
                    "distance": pairwise_dist_matrix[i, j],
                })
            
            ts_distances.append(pairwise_dist_matrix[i, j])
    if len(ts_distances) > 0:
        min_distances.append(min(ts_distances))

In [47]:
len(distances), len(min_distances)

(24010, 24010)

In [48]:
np.array(min_distances).mean()

687.7690054686191

### Running in parallel

In [15]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

In [16]:
def compute_social_distances(time, input_df: pd.DataFrame):
    distances = []
    agents_combinations = list(combinations(group["ag_id"], 2))
    distances_ts = []
    for ag1, ag2 in agents_combinations:
        x1, y1 = group.loc[group["ag_id"] == ag1, ["x", "y"]].values[0]
        x2, y2 = group.loc[group["ag_id"] == ag2, ["x", "y"]].values[0]
        distance = euclidean_distance((x1, y1), (x2, y2))
        if not np.isnan(distance): 
            distances_ts.append(distance)
            distances.append(
                {
                    "timestamp": time,
                    "ag_id1": ag1,
                    "ag_id2": ag2,
                    "distance": distance,
                }
            )
            
    min_distance = min(distances_ts)
    return min_distance, distances

In [17]:
overall_min_distances = []
with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda x: compute_social_distances(x[0], x[1]), grouped_frames))

In [ ]:
overall_distances = []
for min_distances, distances in results:
    overall_min_distances.append(min_distances)
    overall_distances.append(distances)